# MSIN0107, Advanced Quantitative Finance
## Coursework I
### Students: 
- Ionut Nodis 
- Adria Ramoneda
- Rayan Sidani 
- Oleg Yushkevich


#### When handing in, please explain which formula(s) and inputs you have used when solving each question. You are welcome to post code in an Appendix, but it should be possible to understand what you have done based on your written answers alone.

# Question 1

Consider a 1-year put option with strike price 100 on a non-dividend paying tock with the following parameters:
- Stock price: 100 GBP 
- $\sigma$: 20% per annum 
- Risk free rate: 1.5% cont. comp. per annum

Assume that the standard Black-Scholes assumptions apply, i.e. the underlying stock price follows a Geometric Brownian Motion and the interest
rate is constant. Assume the year has 252 business days


#### a) Standard put. Compute the price of a standard put option using Monte Carlo simulation with N = 10, 000 draws and compute a 95% condence interval for the option price.

In [14]:
#Import libraries

import numpy as np 
import matplotlib.pyplot as plt 
from scipy.stats import norm

In [21]:
#random normal distribution 

np.random.seed(1) 

# Parameters
S0 = 100
r = 0.015
T = 1
sigma = 0.20
K = 100
N = 10000

#Put option using Black Scholes Model 

def black_scholes_price(S0, K, T, r, sigma):
    d1 = (np.log(S0/K) + (r + 0.5*sigma**2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    put = K*np.exp(-r*T)*norm.cdf(-d2) - S0*norm.cdf(-d1)
    return put


P_bs = black_scholes_price(S0, K, T, r, sigma)

#Put option price using MC:

# Mean and variance of LogS_T
logS_mean = np.log(S0) + (r - 0.5*sigma**2) * T
logS_std = sigma * np.sqrt(T)

# Simulate Nx1 normal RVs
logS = np.random.normal(logS_mean, logS_std, N) 

# Payoff of put for each simulation
V = np.exp(-r*T) * np.maximum(K - np.exp(logS), 0)

# Value of put
P_mc = np.mean(V) #we take the average of all sampled paths and compute the present price of the option

# Display results
print(f"The MC price using {N} simulations is {P_mc}.")
sigv = np.std(V)
CI = [np.mean(V) - 1.96*sigv/np.sqrt(N), np.mean(V) + 1.96*sigv/np.sqrt(N)]
print(f"A 95% confidence band for the MC price is ({CI[0]}; {CI[1]}).")
print(f"The width of the confidence band is {CI[1] - CI[0]}.")
print(f"The Black-Scholes price is {P_bs}.")
print()


The MC price using 10000 simulations is 7.073851998197918.
A 95% confidence band for the MC price is (6.881734287293494; 7.265969709102343).
The width of the confidence band is 0.3842354218088495.
The Black-Scholes price is 7.184019975124272.



#### b) Compute the return of a standard put option using Monte Carlo simulation with N= 10,000 draws using Antithetic Variables and compute a 95% confidence interval for the option price. Is the confidence interval wider or narrower than that in the previous question? Is the result closer to the theoretical B-S result compared to the previous case?

In [22]:
#Compute the option price using Antithetic Variables 

X = np.random.normal(0, logS_std, N)

logS1 = logS_mean + X
logS2 = logS_mean - X

V_av = 0.5 * (np.maximum(K - np.exp(logS1), 0) + np.maximum(K - np.exp(logS2), 0))
P_av = np.exp(-r*T) * np.mean(V_av)

sigv_av = np.std(V_av)

CI_av_u = np.exp(-r*T) * (np.mean(V_av) + 1.96*sigv_av/np.sqrt(N))
CI_av_l = np.exp(-r*T) * (np.mean(V_av) - 1.96*sigv_av/np.sqrt(N))

print('The MC price using antithetic variables is', P_av)
print('A 95% confidence band for the MC price using antithetic variables is', [CI_av_l, CI_av_u])
print('The width of the confidence band is', CI_av_u - CI_av_l)
print(f"The Black-Scholes price is {P_bs}.")
print()

# Previous question results
CI_standard_u = CI[1]
CI_standard_l = CI[0]

# Compare confidence intervals
width_standard = CI_standard_u - CI_standard_l
width_av = CI_av_u - CI_av_l

if width_av < width_standard:
    print("The confidence interval using antithetic variables is narrower than that in the previous question.")
else:
    print("The confidence interval using antithetic variables is wider than that in the previous question.")

The MC price using antithetic variables is 7.207943466280324
A 95% confidence band for the MC price using antithetic variables is [7.114235553529572, 7.3016513790310755]
The width of the confidence band is 0.18741582550150326
The Black-Scholes price is 7.184019975124272.

The confidence interval using antithetic variables is narrower than that in the previous question.


#### c) Option with knock-in barrier. A knock-in option is activated if the underlying asset reaches a predetermined barrier during its life. Assume that the put has the "barrier" of $80: If the stock price decreases below $80, the option is activated. Compute the price of the put with the barrier using Monte Carlo simulation with N = 10, 000 draws. Is it different from the price of a standard put (why)? Computea 95% condence interval for the option return.

In [23]:
# Parameters
S0 = 100
r = 0.015
T = 1
sigma = 0.20
K = 100
N = 10000
barrier = 80

# Mean and variance of LogS_T
logS_mean = np.log(S0) + (r - 0.5 * sigma**2) * T
logS_std = sigma * np.sqrt(T)

# Simulate Nx1 normal RVs
logS = np.random.normal(logS_mean, logS_std, N)

# Simulate stock paths
dt = T / 252  # daily steps
S_paths = np.zeros((N, 252 + 1))
S_paths[:, 0] = S0
for t in range(1, 252 + 1):
    Z = np.random.normal(0, 1, N)
    S_paths[:, t] = S_paths[:, t - 1] * np.exp((r - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * Z)

# Check if the barrier is hit
knock_in = np.min(S_paths, axis=1) < barrier

# Payoff of put for each simulation
V = np.exp(-r * T) * np.maximum(K - np.exp(logS), 0) * knock_in

# Value of put
P_knock_in = np.mean(V)
sigv_knock_in = np.std(V)
CI_knock_in_u = np.mean(V) + 1.96 * sigv_knock_in / np.sqrt(N)
CI_knock_in_l = np.mean(V) - 1.96 * sigv_knock_in / np.sqrt(N)


#Compute the price of a standard put option using Black Scholes Model
def black_scholes_price(S0, K, T, r, sigma):
    d1 = (np.log(S0/K) + (r + 0.5*sigma**2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    put = K*np.exp(-r*T)*norm.cdf(-d2) - S0*norm.cdf(-d1)
    return put

# Compute the price of the put option
P_bs = black_scholes_price(S0, K, T, r, sigma)


# Display results
print(f"The MC price of the knock-in put option using {N} simulations is {P_knock_in}.")
print(f"A 95% confidence band for the MC price is [{CI_knock_in_l}, {CI_knock_in_u}].")
print(f"The width of the confidence band is {CI_knock_in_u - CI_knock_in_l}.")
print(f"The Black-Scholes price is {P_bs} for the standard put option.")
print()

# Compare the prices
if P_knock_in > P_bs:
    print("The price of the knock-in put option is higher than the price of the standard European put option.")
else:
    print("The price of the knock-in put option is lower than the price of the standard European put option.")

The MC price of the knock-in put option using 10000 simulations is 1.864274942491179.
A 95% confidence band for the MC price is [1.748666349298067, 1.9798835356842912].
The width of the confidence band is 0.23121718638622424.
The Black-Scholes price is 7.184019975124272 for the standard put option.

The price of the knock-in put option is lower than the price of the standard European put option.


#### d) Compute the value of the option with knock-out barrier above using Monte Carlo simulation with N = 10, 000 draws and using the standard put option as a control variate. Also, compute a 95% confidence interval for the option price. Is the confidence interval wider or narrower than that in the previous question?

In [24]:
# Parameters
S0 = 100
r = 0.015
T = 1
sigma = 0.20
K = 100
N = 10000
barrier = 80

# Mean and variance of LogS_T
logS_mean = np.log(S0) + (r - 0.5 * sigma**2) * T
logS_std = sigma * np.sqrt(T)

# Simulate Nx1 normal RVs
logS = np.random.normal(logS_mean, logS_std, N)

# Simulate stock paths
dt = T / 252  # daily steps
S_paths = np.zeros((N, 252 + 1))
S_paths[:, 0] = S0
for t in range(1, 252 + 1):
    Z = np.random.normal(0, 1, N)
    S_paths[:, t] = S_paths[:, t - 1] * np.exp((r - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * Z)

# Check if the barrier is hit
knock_out = np.min(S_paths, axis=1) >= barrier

# Payoff of knock-out put for each simulation
V_knock_out = np.exp(-r * T) * np.maximum(K - np.exp(logS), 0) * knock_out

# Payoff of standard put for each simulation
V_standard = np.exp(-r * T) * np.maximum(K - np.exp(logS), 0)

# Control variate method
P_standard = np.mean(V_standard)
covariance = np.cov(V_knock_out, V_standard)[0, 1]
variance_standard = np.var(V_standard)
c = -covariance / variance_standard

# Adjusted payoff
V_adjusted = V_knock_out + c * (V_standard - P_standard)

# Value of knock-out put using control variate
P_knock_out_cv = np.mean(V_adjusted)
sigv_knock_out_cv = np.std(V_adjusted)
CI_knock_out_cv_u = P_knock_out_cv + 1.96 * sigv_knock_out_cv / np.sqrt(N)
CI_knock_out_cv_l = P_knock_out_cv - 1.96 * sigv_knock_out_cv / np.sqrt(N)

# Display results
print(f"The MC price of the knock-out put option using {N} simulations and control variate is {P_knock_out_cv}.")
print(f"A 95% confidence band for the MC price is [{CI_knock_out_cv_l}, {CI_knock_out_cv_u}].")
print(f"The width of the confidence band is {CI_knock_out_cv_u - CI_knock_out_cv_l}.")

# Compare confidence intervals
width_knock_out = CI_knock_out_cv_u - CI_knock_out_cv_l
width_knock_in = CI_knock_in_u - CI_knock_in_l 

if width_knock_out < width_knock_in:
    print("The confidence interval is narrower than that in the previous question.")
else:
    print("The confidence interval is wider than that in the previous question.")
    

The MC price of the knock-out put option using 10000 simulations and control variate is 5.422610665132544.
A 95% confidence band for the MC price is [5.319007661740143, 5.526213668524946].
The width of the confidence band is 0.20720600678480317.
The confidence interval is narrower than that in the previous question.


# Question 2

Assume that the final payoff of the option is $max(K-S, 0)$ where $K$ is the maximal stock price during the life of the option ($K = max\{S_i\}$), where $S_i$ is the stock price on day $i$. The option has no barrier. This is a "Lookback Put":
The strike price is the highest price of the underlying asset over the option's life, allowing the holder to sell at the highest price (the option is path-dependent). 


#### a) Compute the price of the Lookback put option using Monte Carlo simulation with N = 10, 000 draws. Also, compute a 95% confidence interval for the option price. Is the price higher or lower than the value of the standard put option above with the given strike price K = 100? Why?

In [25]:
# Parameters
S0 = 100
r = 0.015
T = 1
sigma = 0.20
N = 10000

# Simulate stock paths
dt = T / 252  # daily steps
S_paths = np.zeros((N, 252 + 1))
S_paths[:, 0] = S0
for t in range(1, 252 + 1):
    Z = np.random.normal(0, 1, N)
    S_paths[:, t] = S_paths[:, t - 1] * np.exp((r - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * Z)

# Determine the maximum stock price during the life of the option
K_max = np.max(S_paths, axis=1)

# Payoff of Lookback put for each simulation
V_lookback = np.exp(-r * T) * np.maximum(K_max - S_paths[:, -1], 0)

# Value of Lookback put
P_lookback = np.mean(V_lookback)
sigv_lookback = np.std(V_lookback)
CI_lookback_u = P_lookback + 1.96 * sigv_lookback / np.sqrt(N)
CI_lookback_l = P_lookback - 1.96 * sigv_lookback / np.sqrt(N)

# Display results
print(f"The MC price of the Lookback put option using {N} simulations is {P_lookback}.")
print(f"A 95% confidence band for the MC price is [{CI_lookback_l}, {CI_lookback_u}].")
print(f"The width of the confidence band is {CI_lookback_u - CI_lookback_l}.")

# Compare with the standard put option price
P_standard = np.mean(np.exp(-r * T) * np.maximum(100 - S_paths[:, -1], 0))  # Assuming K = 100 for standard put
if P_lookback > P_standard:
    print("The price of the Lookback put option is higher than the value of the standard put option.")
else:
    print("The price of the Lookback put option is lower than the value of the standard put option.")
    

The MC price of the Lookback put option using 10000 simulations is 15.447870271083486.
A 95% confidence band for the MC price is [15.239145883936576, 15.656594658230397].
The width of the confidence band is 0.41744877429382043.
The price of the Lookback put option is higher than the value of the standard put option.


#### b) You want the price of the Lookback option to be suciently precise. Specifically, you want the width of the 95% confidence band 10 to be £0.01, i.e. if the confidence band is [c1; c2] then c2−c1 = 0.01. How many simulations N do you need to achieve this when doing standard Monte Carlo simulation? How many simulations N do you need to achieve this when doing Monte Carlo simulation using Antithetic Variables

In [27]:
# Parameters
S0 = 100
r = 0.015
T = 1
sigma = 0.20
N_initial = 10000  # Initial number of simulations to estimate sigma

# Simulate stock paths
dt = T / 252  # daily steps
S_paths = np.zeros((N_initial, 252 + 1))
S_paths[:, 0] = S0
for t in range(1, 252 + 1):
    Z = np.random.normal(0, 1, N_initial)
    S_paths[:, t] = S_paths[:, t - 1] * np.exp((r - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * Z)

# Determine the maximum stock price during the life of the option
K_max = np.max(S_paths, axis=1)

# Payoff of Lookback put for each simulation
V_lookback = np.exp(-r * T) * np.maximum(K_max - S_paths[:, -1], 0)

# Estimate standard deviation
sigma_lookback = np.std(V_lookback)

# Calculate required N for standard Monte Carlo
desired_width = 0.01
N_standard = (2 * 1.96 * sigma_lookback / desired_width) ** 2

# Using Antithetic Variables
X = np.random.normal(0, sigma * np.sqrt(T), N_initial)
logS1 = logS_mean + X
logS2 = logS_mean - X
V_av = 0.5 * (np.maximum(K_max - np.exp(logS1), 0) + np.maximum(K_max - np.exp(logS2), 0))
sigma_av = np.std(V_av)

# Calculate required N for Monte Carlo with Antithetic Variables
N_antithetic = (2 * 1.96 * sigma_av / desired_width) ** 2

# Display results
print(f"Estimated standard deviation for Lookback put: {sigma_lookback}")
print(f"Number of simulations needed for standard Monte Carlo: {int(np.ceil(N_standard))}")
print(f"Estimated standard deviation for Lookback put using Antithetic Variables: {sigma_av}")
print(f"Number of simulations needed for Monte Carlo with Antithetic Variables: {int(np.ceil(N_antithetic)):.0f}")

Estimated standard deviation for Lookback put: 10.429684619131377
Number of simulations needed for standard Monte Carlo: 16715312
Estimated standard deviation for Lookback put using Antithetic Variables: 12.930795317360495
Number of simulations needed for Monte Carlo with Antithetic Variables: 25693461
